**Niks Pille**

All Tickers

In [9]:
#Getting all tickers in a list
All_Tickers = []
for i in range(4521):
    All_Tickers.append(Only_Common_Stocks['symbol'][i])

Creating 'Stock_With_All_Data'

In [171]:
Stocks_With_All_Data = []
for i in range(4521):
    if len(yf.Ticker(All_Tickers[i]).history(start='2000-01-02', end='2020-01-01')[['Close']]) == 5031:
        Stocks_With_All_Data.append(All_Tickers[i])

- AGM.A: No data found, symbol may be delisted
- ALUS: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- AMHC: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- ARQT: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BATL: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BDTX: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BEAM: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BF.A: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- BH.A: No data found, symbol may be delisted
- BIO.B: No data found for this date range, symbol may be delisted
- BRK.A: No data found, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- BSBK: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BWL.A: No data found, symbol may be delisted
- CHPM: Data doesn't exist for startDate = 946767600, endDate 

In [186]:
with open('Stocks_With_Data.csv','w',newline='') as file:
    wr = csv.writer(file, quoting=csv.QUOTE_ALL)
    wr.writerow(Stocks_With_All_Data)

Creating 'Three_Month_Treasury_Bill'

In [38]:
#DTB3 is taken from https://fred.stlouisfed.org/
DTB3 = pd.read_csv('DTB3.csv', index_col = 0)
#Using Apple as a benchmark
aapl = yf.Ticker('aapl').history(start='2000-01-02', end='2020-01-01')[['Close']]
#Getting dates from Apple
Dates = []
for i in range(len(aapl)):
    if str(aapl.index[i])[:10] in DTB3.index:
        Dates.append(str(aapl.index[i])[:10])
#Sorting out missing values
Three_Month_Treasury_Bill = DTB3.loc[Dates][DTB3.loc[Dates].DTB3 != '.']
#Creating a csv-file with the Three Month Treasury Bill
Three_Month_Treasury_Bill.to_csv('Three_Month_Treasury_Bill.csv')

Creating 'All_Stocks'

In [45]:
#Make a table with the new dates
Ny = DTB3.loc[Dates]
#Make a new column with strings
Ny['Datoer'] = Dates
#Set the new column as index to make it a callable string
Nyy = Ny.set_index('Datoer')
#A for loop to get all tables for the chosen stocks into a single list, to be used in concatenating
stock = []
for i in list(dict.fromkeys(Stocks)):
    asset = yf.Ticker(i).history(start='2000-01-02', end='2020-01-01')[['Close']]
    asset.columns = [i]
    stock.append(asset)
#Concat all the tables
Concat_Table = pd.concat([i for i in stock], axis=1, join='inner')
#Concat the stocks with the risk free rate
All_Tables = pd.concat([Nyy, Concat_Table.reindex(Nyy.index)],axis=1)
#Save to a csv file
All_Tables.to_csv('All_Stocks.csv')

Creating 'Table'

In [48]:
Everything = pd.read_csv('All_Stocks.csv', index_col=0)
#Russell3000 is used as the market rate since it covers 98% of the American market
Russell_3000 = yf.Ticker('^RUA').history(start='2000-01-02', end='2020-01-01')[['Close']]
#Sorting out missing values from the risk-free rate
gj = Everything.loc[:][Everything.loc[:].DTB3 != '.']
#Making a new date list with the dates from Russell 3000
Datess = []
for i in range(len(Russell_3000)):
    if str(Russell_3000.index[i])[:10] in gj.index:
        Datess.append(str(Russell_3000.index[i])[:10])
#Making the dates strings so they are callable
gh = []
for i in range(len(Russell_3000)):
    gh.append(str(Russell_3000.index[i])[:10])
#Making a new column with the string dates and set it as index
Russell_3000['Datoer'] = gh
gk = Russell_3000.set_index('Datoer')
#Set the name of the column
gk.columns = ['Russell3000']
#Concat Russell3000 to the rest
Table = pd.concat([gj.loc[Datess],gk.loc[Datess]],axis=1)
Table.to_csv('Table.csv')

# Bachelorproject

Documentation:  
https://addisonlynch.github.io/iexfinance/stable/stocks.html  
https://github.com/ranaroussi/yfinance

## Getting Data

In [1]:
#All packages used
from iexfinance.stocks import Stock
from iexfinance.refdata import get_symbols
import yfinance as yf
import numpy as np
import pandas as pd
import csv
token = 'sk_32c5d1b5f6b044f294391ae88eac3e47'

In [3]:
#Get a table with everything on the financial markets in USA
Alle_symboler = get_symbols(output_format='pandas', token=token)
#Sorting out all the non common stocks
Only_Common_Stocks = get_symbols()[get_symbols().type == 'cs'].reset_index(drop=True)

In [4]:
with open('Stocks_With_Data.csv',newline='') as file:
    reader = csv.reader(file)
    Stocks_With_All_Data = list(reader)
#Drawing 300 random stocks for which to be analyzed
Stocks = []
np.random.seed(2020)
for i in range(300):
    Stocks.append(Stocks_With_All_Data[0][np.random.randint(0,len(Stocks_With_All_Data[0]))])

Stocks.sort()
Every_Stock = list(dict.fromkeys(Stocks))

### Adjusting so all data match with no missing values

In [21]:
All = pd.read_csv('Table.csv', index_col=0)

In [22]:
columns = []
for i in range(283):
    columns.append(All.columns[i])
#Creating a table with the log values
Logtable = pd.DataFrame()
for i in range(283):
    Logtable[columns[i]] = All[columns[i]].apply(np.log)

In [23]:
Daily = pd.DataFrame()
for i in range(len(columns)):
    Daily[columns[i]] = np.diff(Logtable[columns[i]],axis=0)

C:\Users\xande\Anaconda3\lib\site-packages\numpy\lib\function_base.py:1273: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])


In [24]:
Daily_returns = Daily.set_index(Logtable.index.values[0:-1])
a = ['NaN','inf', '-inf']
Daily_returnss = Daily_returns[~Daily_returns['DTB3'].isin(a)].dropna()
Daily_returnss.to_csv('Daily_returns.csv')

In [25]:
Daily_returns = pd.read_csv('Daily_returns.csv', index_col=0)

In [26]:
Excess_return = pd.DataFrame()

for i in range(len(columns)):
    Excess_return[columns[i]] = Daily_returns[columns[i]] - Daily_returns['DTB3']
del Excess_return['DTB3']

In [27]:
Excess_return.to_csv('Excess_return.csv')

In [2]:
list = []
for i in range(284,565,1):
    list.append(i)
DCC = pd.read_csv('DCC_ENGLE_ESTIMATES.csv', usecols=list)

In [3]:
columns = []
for i in range(280):
    columns.append(DCC.columns[i])
DCC_Betas = pd.DataFrame()
for i in range(len(columns)):
    DCC_Betas[str(columns[i][16:])] = DCC[columns[i]]/DCC['CondVar_Russell3000']

In [4]:
DCC_Betas

,A,AB,ABC,AEE,AEM,AES,AGM,AIT,AJG,AMRB,...,WTBA,WTM,WVVI,WWD,WWE,WYY,XLNX,XRAY,XRX,Y
0,0.963461,0.726861,0.937743,0.993415,0.973732,0.903934,0.574007,0.954319,0.981775,0.928080,...,0.951051,0.707107,0.938068,0.661624,0.535313,0.957066,0.892779,0.951069,0.814851,-0.141580
1,0.983792,0.730634,0.952834,0.999637,1.003371,0.922245,0.580985,0.983155,0.990680,0.947463,...,0.974640,0.709138,0.962510,0.676917,0.553531,1.000559,0.902929,0.958395,0.827866,-0.142626
2,1.004552,0.734427,0.968168,1.005898,1.033912,0.940927,0.588048,1.012863,0.999666,0.967252,...,0.998814,0.711174,0.987590,0.692563,0.572369,1.046029,0.913194,0.965778,0.841088,-0.143679
3,1.025750,0.738239,0.983749,1.012198,1.065383,0.959987,0.595196,1.043468,1.008734,0.987454,...,1.023588,0.713216,1.013323,0.708571,0.591847,1.093565,0.923577,0.973217,0.854521,-0.144741
4,1.047395,0.742072,0.999580,1.018537,1.097812,0.979433,0.602432,1.074998,1.017884,1.008078,...,1.048976,0.715264,1.039727,0.724949,0.611989,1.143261,0.934077,0.980714,0.868169,-0.145810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4930,1.059108,1.342516,1.200282,1.329598,0.719155,0.877374,1.455164,1.400515,1.347076,1.327793,...,1.067479,0.687938,0.771108,1.061217,0.617527,1.046195,1.164292,1.070143,1.089241,1.156656
4931,1.085612,1.360909,1.219575,1.386509,0.996041,0.898617,1.469200,1.517512,1.457646,1.419995,...,1.104748,0.793528,0.830437,1.062280,0.639841,1.011345,1.173790,1.104955,1.122275,1.137603
4932,1.038483,1.218167,1.076426,1.270247,0.980645,0.982023,1.396857,1.464699,1.332873,1.317076,...,1.049811,0.609170,0.873733,0.914666,0.752109,0.687205,1.063416,0.937523,1.094454,1.063856
4933,1.058926,0.966971,1.082160,1.487341,1.028262,1.014079,1.339006,1.453938,1.393280,1.345509,...,1.247897,0.513572,0.930126,0.914990,0.671493,1.277056,1.041394,1.116476,1.063777,1.086836


In [5]:
DCC_Betas.iloc[4934][1]

0.8484751952375358

In [6]:
pd.qcut(DCC_Betas.iloc[0],10,labels=False)

A       7
AB      2
ABC     5
AEE     9
AEM     8
       ..
WYY     7
XLNX    3
XRAY    6
XRX     3
Y       0
Name: 0, Length: 280, dtype: int64

In [84]:
zero  = []
one   = []
two   = []
three = []
four  = []
five  = []
six   = []
seven = []
eight = []
nine  = []

In [85]:
for i in range(len(DCC_Betas)):
    for j in range(len(columns)):
        if pd.qcut(DCC_Betas.iloc[i],10,labels=False).iloc[j] == 0:
            zero.append(DCC_Betas.iloc[i][j])
        elif pd.qcut(DCC_Betas.iloc[i],10,labels=False).iloc[j] == 1:
            one.append(DCC_Betas.iloc[i][j])
        elif pd.qcut(DCC_Betas.iloc[i],10,labels=False).iloc[j] == 2:
            two.append(DCC_Betas.iloc[i][j])
        elif pd.qcut(DCC_Betas.iloc[i],10,labels=False).iloc[j] == 3:
            three.append(DCC_Betas.iloc[i][j])
        elif pd.qcut(DCC_Betas.iloc[i],10,labels=False).iloc[j] == 4:
            four.append(DCC_Betas.iloc[i][j])
        elif pd.qcut(DCC_Betas.iloc[i],10,labels=False).iloc[j] == 5:
            five.append(DCC_Betas.iloc[i][j])
        elif pd.qcut(DCC_Betas.iloc[i],10,labels=False).iloc[j] == 6:
            six.append(DCC_Betas.iloc[i][j])
        elif pd.qcut(DCC_Betas.iloc[i],10,labels=False).iloc[j] == 7:
            seven.append(DCC_Betas.iloc[i][j])
        elif pd.qcut(DCC_Betas.iloc[i],10,labels=False).iloc[j] == 8:
            eight.append(DCC_Betas.iloc[i][j])
        else:
            nine.append(DCC_Betas.iloc[i][j])